<a href="https://colab.research.google.com/github/kholiqhidayat/UAS-Praktikum-Pengolahan-Citra-PCD/blob/main/Project_G_231_22_0097.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, applications
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Load dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

# Normalisasi
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

# Flatten labels
y_train = y_train.flatten()
y_test = y_test.flatten()

# Split validation
x_train, x_val, y_train, y_val = train_test_split(
    x_train, y_train, test_size=0.1, random_state=42
)

print("Train:", x_train.shape)
print("Val:", x_val.shape)
print("Test:", x_test.shape)

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
Train: (45000, 32, 32, 3)
Val: (5000, 32, 32, 3)
Test: (10000, 32, 32, 3)


In [ ]:
from tensorflow import keras
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Load dataset CIFAR-10
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

# Normalisasi
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

# Flatten label
y_train = y_train.flatten()
y_test = y_test.flatten()

# Split menjadi train & val
x_train, x_val, y_train, y_val = train_test_split(
    x_train, y_train, test_size=0.1, random_state=42
)

print("Train:", x_train.shape)
print("Val:", x_val.shape)
print("Test:", x_test.shape)

Train: (45000, 32, 32, 3)
Val: (5000, 32, 32, 3)
Test: (10000, 32, 32, 3)


In [ ]:
IMG_SIZE = 160  # MobileNetV2 minimal 96 px

train_datagen = keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=keras.applications.mobilenet_v2.preprocess_input,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

val_datagen = keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=keras.applications.mobilenet_v2.preprocess_input
)

# PENTING: pakai "flow" BIASA dengan target_size
train_flow = train_datagen.flow(
    x_train, y_train,
    batch_size=32,
    shuffle=True
)

val_flow = val_datagen.flow(
    x_val, y_val,
    batch_size=32,
    shuffle=False
)

print("Data generator siap.")

Data generator siap.


In [ ]:
import tensorflow as tf
from tensorflow import keras

IMG_SIZE = 160

train_datagen = keras.preprocessing.image.ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

val_datagen = keras.preprocessing.image.ImageDataGenerator()

train_flow = train_datagen.flow(
    x_train, y_train,
    batch_size=32,
    shuffle=True
)

val_flow = val_datagen.flow(
    x_val, y_val,
    batch_size=32,
    shuffle=False
)

print("Flow siap digunakan!")

Flow siap digunakan!


In [ ]:
inputs = keras.Input(shape=(32, 32, 3))

# Resize CIFAR-10 ke 160x160 agar MobileNetV2 bisa menerima
x = keras.layers.Resizing(160, 160)(inputs)

# Preprocessing MobileNetV2
x = keras.applications.mobilenet_v2.preprocess_input(x)

# Load MobileNetV2 (tanpa top)
base_model = keras.applications.MobileNetV2(
    input_shape=(160, 160, 3),
    include_top=False,
    weights='imagenet'
)

base_model.trainable = False  # freeze pretrained weights

x = base_model(x, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.3)(x)
outputs = keras.layers.Dense(10, activation='softmax')(x)

model = keras.Model(inputs, outputs)
model.summary()


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resizing (Resizing)             │ (None, 160, 160, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ true_divide (TrueDivide)        │ (None, 160, 160, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ subtract (Subtract)             │ (None, 160, 160, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_160            │ (None, 5, 5, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │        12,810 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,270,794 (8.66 MB)

 Trainable params: 12,810 (50.04 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

In [ ]:
callbacks_list = [
    keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=5,
        restore_best_weights=True
    ),
    keras.callbacks.ModelCheckpoint(
        "mobilenet_best.h5",
        save_best_only=True
    )
]

history = model.fit(
    train_flow,
    validation_data=val_flow,
    epochs=10,
    callbacks=callbacks_list
)

Epoch 1/10


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1406/1407 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.1190 - loss: 2.3782

1407/1407 ━━━━━━━━━━━━━━━━━━━━ 72s 39ms/step - accuracy: 0.1190 - loss: 2.3781 - val_accuracy: 0.1838 - val_loss: 2.2621
Epoch 2/10
1406/1407 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.1499 - loss: 2.2583

1407/1407 ━━━━━━━━━━━━━━━━━━━━ 37s 27ms/step - accuracy: 0.1499 - loss: 2.2583 - val_accuracy: 0.2004 - val_loss: 2.1865
Epoch 3/10
1405/1407 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.1640 - loss: 2.2323

...

1407/1407 ━━━━━━━━━━━━━━━━━━━━ 37s 26ms/step - accuracy: 0.1688 - loss: 2.2156 - val_accuracy: 0.2166 - val_loss: 2.1420
Epoch 5/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 37s 26ms/step - accuracy: 0.1736 - loss: 2.2052 - val_accuracy: 0.2180 - val_loss: 2.1519
Epoch 6/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 36s 26ms/step - accuracy: 0.1782 - loss: 2.2011 - val_accuracy: 0.1936 - val_loss: 2.1482
Epoch 7/10
1406/1407 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.1863 - loss: 2.1859

1407/1407 ━━━━━━━━━━━━━━━━━━━━ 37s 26ms/step - accuracy: 0.1863 - loss: 2.1859 - val_accuracy: 0.2238 - val_loss: 2.1173
Epoch 8/10
1406/1407 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.1938 - loss: 2.1795

1407/1407 ━━━━━━━━━━━━━━━━━━━━ 37s 26ms/step - accuracy: 0.1938 - loss: 2.1795 - val_accuracy: 0.2348 - val_loss: 2.1121
Epoch 9/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.1961 - loss: 2.1761

1407/1407 ━━━━━━━━━━━━━━━━━━━━ 36s 26ms/step - accuracy: 0.1961 - loss: 2.1761 - val_accuracy: 0.2028 - val_loss: 2.1116
Epoch 10/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.1919 - loss: 2.1703

1407/1407 ━━━━━━━━━━━━━━━━━━━━ 36s 26ms/step - accuracy: 0.1919 - loss: 2.1703 - val_accuracy: 0.2706 - val_loss: 2.1111


In [ ]:
x_test_resized = tf.image.resize(x_test, (160, 160))

test_loss, test_acc = model.evaluate(x_test_resized, y_test)
print("Test Accuracy :", test_acc)

313/313 ━━━━━━━━━━━━━━━━━━━━ 20s 51ms/step - accuracy: 0.2754 - loss: 2.0987
Test Accuracy : 0.27059999108314514


In [ ]:
model.save("Project_NIM_G.231.22.0097.h5")
print("Model saved! (Project_NIM_G.231.22.0097.h5)")

Model saved! (Project_NIM_G.231.22.0097.h5)
